In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('weatherAUS.csv')

In [3]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
data.Location.unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

выбираю город, про которых данных больше всего

In [5]:
city = data.groupby(by = 'Location')['Date'].count().sort_values(ascending = False).index[0]

In [6]:
data = data[data.Location == city]

In [7]:
data.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

создаю штуку, которую буду предсказывать -- среднюю температуру по дню

In [8]:
data['MeanTemp'] = (data['MinTemp']+data['MaxTemp'])/2

In [9]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

In [10]:
data = data[['Date', 'MeanTemp']]

меняю пропуски на температуру с предыдущего дня (если предыдущий день -- пропуск, то с дня до):

In [11]:
while sum(data.MeanTemp.isna())!=0:
    ind = data[data.MeanTemp.isna()].index
    data.loc[ind, 'MeanTemp'] = list(data.loc[ind - 1].MeanTemp)

In [12]:
data.set_index('Date', inplace = True)

In [13]:
data

,MeanTemp
Date,
2007-11-01,16.15
2007-11-02,20.45
2007-11-03,18.55
2007-11-04,14.40
2007-11-05,11.85
...,...
2017-06-21,6.15
2017-06-22,6.75
2017-06-23,5.20


In [14]:
rng = pd.date_range(start = '2007-11-01', end = '2017-06-25', freq='D')

In [15]:
missing = pd.DataFrame(set(rng).difference(set(data.index))).set_index(0)

пропущены апрель и декабрь 2012, февраль 2013. заменим апрель и декабрь 2012 значениями с апреля и декабря 2011, а февраль 2013 февралем 2012

In [16]:
missing['MeanTemp'] = data.loc[missing.index - pd.DateOffset(years=1)]

In [17]:
data = data.append(missing)

In [18]:
data.to_csv('canberra_temp.csv')